In [1]:
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow_hub as hub
import numpy as np
%matplotlib inline

In [21]:
df = pd.read_csv("data/question_embeddings0.csv")
df

,0,1,2,3,4,5,6,7,8,9,...,502,503,504,505,506,507,508,509,510,511
0,-0.004167,0.003991,-0.063261,0.051590,-0.041000,-0.044420,0.084289,-0.007363,-0.021403,-0.022497,...,0.060429,-0.062331,-0.017937,-0.033932,0.009563,-0.063174,0.053774,-0.043223,-0.044161,-0.066338
1,0.028851,-0.046769,-0.003425,0.002011,0.038640,0.033116,0.003396,0.041565,-0.004693,0.022073,...,0.045903,0.027739,-0.002885,0.001373,-0.040582,-0.004657,0.026895,0.052670,-0.035369,0.005745
2,-0.027575,0.065745,-0.046614,-0.042876,-0.006606,-0.022570,-0.026257,0.051032,-0.028213,0.030517,...,0.041123,-0.010204,-0.030535,-0.020685,0.019016,-0.022978,0.052545,-0.002048,-0.041150,-0.066599
3,0.011732,0.020154,-0.052613,0.025233,-0.056231,-0.056353,0.091451,-0.017706,-0.005750,0.044669,...,0.058850,-0.002576,-0.043297,-0.069057,-0.025404,-0.071836,0.048371,0.016060,-0.055275,-0.078401
4,0.015800,0.015869,-0.042117,-0.056513,0.059205,-0.003981,0.005216,0.066134,0.032295,0.068357,...,0.025379,0.054186,-0.069469,-0.017845,-0.021630,-0.029128,0.047190,-0.039558,-0.042111,-0.001511
5,-0.031953,-0.002469,-0.041008,0.024304,0.053117,0.003803,0.065083,0.057249,-0.002224,0.045139,...,-0.066791,-0.054315,-0.067143,-0.014493,0.037110,-0.033795,0.052651,-0.053113,-0.047084,-0.069884
6,-0.017040,0.052235,-0.043022,-0.027080,0.010462,0.034135,0.075223,0.039218,-0.028911,0.070063,...,-0.069462,-0.056398,0.021440,-0.032532,0.060512,-0.041380,0.026204,0.077931,-0.025626,0.069311
7,-0.032157,0.034428,-0.023454,0.008774,0.013414,0.075705,0.031538,0.043344,0.032998,0.000894,...,0.019505,0.003753,-0.050828,-0.003165,0.018650,-0.041072,0.038587,0.047902,-0.025329,-0.066533
8,0.008289,-0.024881,-0.014929,-0.015841,0.023373,0.045953,0.019825,-0.009497,0.038823,-0.039870,...,-0.067448,0.071697,-0.036793,-0.029809,0.008289,-0.017808,0.032799,-0.009106,-0.006607,0.026956
9,-0.018821,0.017218,-0.001502,0.013991,0.001999,0.042778,0.016658,0.035108,0.007042,0.000420,...,0.052234,-0.030651,-0.011041,0.021505,-0.062632,-0.047907,0.026807,-0.065929,-0.047524,0.014051


In [30]:
# read training data
train_df = pd.read_csv("data/train.csv")
print("Training Data Frame Dimensions: " + str(train_df.shape))

Training Data Frame Dimensions: (1306122, 3)


In [12]:
test = np.array([1,2,3,4])

In [18]:
test[2:4]

array([3, 4])

In [31]:
# first 5 elements of training data
train_df.head()

,qid,question_text,target
0,00002165364db923c7e6,How did Quebec nationalists see their province...,0
1,000032939017120e6e44,"Do you have an adopted dog, how would you enco...",0
2,0000412ca6e4628ce2cf,Why does velocity affect time? Does velocity a...,0
3,000042bf85aa498cd78e,How did Otto von Guericke used the Magdeburg h...,0
4,0000455dfa3e01eae3af,Can I convert montra helicon D to a mountain b...,0


In [32]:
# last 5 elements of training data
train_df.tail()

,qid,question_text,target
1306117,ffffcc4e2331aaf1e41e,What other technical skills do you need as a c...,0
1306118,ffffd431801e5a2f4861,Does MS in ECE have good job prospects in USA ...,0
1306119,ffffd48fb36b63db010c,Is foam insulation toxic?,0
1306120,ffffec519fa37cf60c78,How can one start a research project based on ...,0
1306121,ffffed09fedb5088744a,Who wins in a battle between a Wolverine and a...,0


In [33]:
insincere_df = train_df.loc[train_df["target"] == 1]
print("The number of insincere is " + str(insincere_df.shape[0]))

The number of insincere is 80810


In [34]:
sincere_df = train_df.loc[train_df["target"] == 0]
print("The number of sincere is " + str(sincere_df.shape[0]))

The number of sincere is 1225312


In [42]:
# get numpy array of questions
questions = train_df["question_text"].values

In [57]:
test = questions[0:2]
test

array(['How did Quebec nationalists see their province as a nation in the 1960s?',
       'Do you have an adopted dog, how would you encourage people to adopt and not shop?'],
      dtype=object)

In [58]:
# Import the Universal Sentence Encoder's TF Hub module
module_url = "https://tfhub.dev/google/universal-sentence-encoder/2"
embed = hub.Module(module_url)

In [63]:
with tf.Session() as sess:
    sess.run([tf.global_variables_initializer(), tf.tables_initializer()])
    question_embeddings = sess.run(embed(test))
    print(question_embeddings)
    print(question_embeddings.shape)
    embedding_df = pd.DataFrame(question_embeddings)
    embedding_df.to_csv("test.csv", index=False)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore
[[-0.00416658  0.00399125 -0.06326147 ... -0.04322257 -0.04416081
  -0.06633756]
 [ 0.028851   -0.04676896 -0.00342507 ...  0.05266976 -0.03536891
   0.00574462]]
(2, 512)


In [64]:
pd.read_csv("test.csv")

,0,1,2,3,4,5,6,7,8,9,...,502,503,504,505,506,507,508,509,510,511
0,-0.004167,0.003991,-0.063261,0.051590,-0.04100,-0.044420,0.084289,-0.007363,-0.021403,-0.022497,...,0.060429,-0.062331,-0.017937,-0.033932,0.009563,-0.063174,0.053774,-0.043223,-0.044161,-0.066338
1,0.028851,-0.046769,-0.003425,0.002011,0.03864,0.033116,0.003396,0.041565,-0.004693,0.022073,...,0.045903,0.027739,-0.002885,0.001373,-0.040582,-0.004657,0.026895,0.052670,-0.035369,0.005745


In [67]:
tf.__version__

'1.10.0'

In [68]:
pd.read_csv("test_real.csv")

,0,1,2,3,4,5,6,7,8,9,...,502,503,504,505,506,507,508,509,510,511
0,-0.004167,0.003991,-0.063261,0.051590,-0.04100,-0.044420,0.084289,-0.007363,-0.021403,-0.022497,...,0.060429,-0.062331,-0.017937,-0.033932,0.009563,-0.063174,0.053774,-0.043223,-0.044161,-0.066338
1,0.028851,-0.046769,-0.003425,0.002011,0.03864,0.033116,0.003396,0.041565,-0.004693,0.022073,...,0.045903,0.027739,-0.002885,0.001373,-0.040582,-0.004657,0.026895,0.052670,-0.035369,0.005745


In [82]:
from keras.models import Model
from keras.layers import Input, Dense, Activation

In [83]:
X = Input(shape=(512,))
hidden1 = Dense(units=1024, activation='relu')(X)
hidden2 = Dense(units=512, activation='relu')(hidden1)
hidden3 = Dense(units=256, activation='relu')(hidden2)
output = Dense(units=1, activation='sigmoid')(hidden3)
model = Model(inputs=X, outputs=output)
model.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 512)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 1024)              525312    
_________________________________________________________________
dense_6 (Dense)              (None, 512)               524800    
_________________________________________________________________
dense_7 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 257       
Total params: 1,181,697
Trainable params: 1,181,697
Non-trainable params: 0
_________________________________________________________________


In [84]:
# generate random training data
x_train = np.random.random((100000, 512))
y_train = np.random.randint(2, size=(100000, 1))

In [ ]:
model.fit(x_train, y_train, epochs=20)

Epoch 1/20
100000/100000 [==============================] - 51s 515us/step - loss: 0.6938 - acc: 0.4999
Epoch 2/20
100000/100000 [==============================] - 40s 404us/step - loss: 0.6932 - acc: 0.4995
Epoch 3/20
100000/100000 [==============================] - 40s 404us/step - loss: 0.6933 - acc: 0.4990
Epoch 4/20
 17728/100000 [====>.........................] - ETA: 32s - loss: 0.6932 - acc: 0.5013

In [78]:
np.sum((model.predict(x_train) >= 0.5) == y_train)

910

In [80]:
(model.predict(x_train) >= 0.5)[0:5]

array([[ True],
       [ True],
       [False],
       [False],
       [ True]])

In [81]:
y_train[0:5]

array([[1],
       [1],
       [0],
       [0],
       [1]])